In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Autenticación para Google Colab
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '16H9nKWEaF2iB7eoVHLN9wEWg2T63Y9PE'

# Función para obtener archivos de una carpeta, incluyendo información de subcarpetas
def get_files_from_folder(folder_id, folder_name=''):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    files_info = []
    for file in file_list:
        if file['mimeType'] == 'application/vnd.google-apps.folder':  # Es una subcarpeta
            subfolder_name = f"{folder_name}/{file['title']}" if folder_name else file['title']
            files_info.extend(get_files_from_folder(file['id'], subfolder_name))
        else:
            files_info.append({'folder': folder_name, 'title': file['title'], 'link': file['webContentLink']})
    return files_info

# Obtener lista de archivos
files = get_files_from_folder(folder_id)

df = pd.DataFrame(files, columns=['folder', 'title', 'link'])

# Renombrar columnas
df.rename(columns={'folder': 'Nombre carpeta', 'title': 'Nombre imagen', 'link': 'Ruta imagen'}, inplace=True)

# Guardar el DataFrame como CSV
df.to_csv('imagenes.csv', index=False)

# Descargar el archivo CSV en la máquina local
from google.colab import files
files.download('imagenes.csv')
